# 大纲
1. props
2. 事件(events)
3. Fallthrough Attributes
4. Slots
5. Provide/inject
6. Async Components

# props
## prop参数声明
prop属性用于给子组件**实例**传递参数，需要在组件的props选项中声明
- 该声明可以类比构造函数的形参，用于构造（实例化）子组件实例
- 我们也可以称prop属性为prop参数

```js
export default {
  props: ['foo'],  // prop参数声明，一个字符串数组，每个字符串是一个参数名称
  created() {
    // props are exposed on `this`
    console.log(this.foo)
  }
}
```
在组件中，需要用`this`来访问该参数，因为该参数是组件实例的，不是组件的 (类比在c++的类方法中访问成员变量）

除了字符串数组外，props选项还可以是一个Object
```js
export default {
  props: {
    title: String,
    likes: Number
  }
}
```
该props Object中，每个属性对应一个参数
- 属性名称是参数名称，属性值是参数类型（本质上是参数类型的构造函数）
- 参数类型可以使得组件的参数更容易理解，还可以做prop类型验证（使用者传递错误的参数类型会触发警告）

## prop参数传递细节
prop参数是一个字符串，理论上任意的字符串都可以用。为了方便，一般用camelCase(驼峰法）命名。

```js
export default {
  props: {
    greetingMessage: String //声明时，作为属性名称，驼峰法名称写起来方便，不需要双引号
  }
}
```
```html
<span>{{ greetingMessage }}</span>

<!-- 在非In-DOM写法中，驼峰法名称也可以直接写，不过习惯上还是会转换成连字符法（在In-DOM写法中则必须转换）-->
<MyComponent greeting-message="hello" /> 
```

在可以的情况下，组件名称标签尽量用PascalCase写法，并且写成自关闭标签，这样可以提高易读性，很容易区分自定义Vue组件和原生的html元素。但是参数传递没有这样的好处。

## 静态 vs. 动态 props
参数传递是在组件实例化时，以设置html属性的方式进行的。既然是html属性设置，那么可以是静态，也可以是动态
- 静态时会解析成字符串
- 动态时会解析成JS表达式（将引号内部看作JS代码）

```html
<BlogPost title="My journey with Vue" />

<!-- Dynamically assign the value of a variable -->
<BlogPost :title="post.title" />

<!-- Dynamically assign the value of a complex expression -->
<BlogPost :title="post.title + ' by ' + post.author.name" />
```

## 参数类型
参数的类型不限于字符串类型，可以是任意JS类型，例如布尔类型
```html
<!-- 参数若是没有值，默认为 `true`. -->
<BlogPost is-published />

<!--  注意这里需要用动态绑定，否则会解析成字符串 -->
<BlogPost :is-published="false" />

<!-- 动态绑定到某个变量上 -->
<BlogPost :is-published="post.isPublished" />
```

例如数组、对象
```html
<BlogPost :comment-ids="[234, 266, 273]" />

<BlogPost :comment-ids="post.commentIds" />
<BlogPost
  :author="{
    name: 'Veronica',
    company: 'Veridian Dynamics'
  }"
 />

<BlogPost :author="post.author" />
```

`v-bind`（动态绑定）可以不带参数，用来同时绑定多个属性（之前学的是html自带属性，这里的prop属性也可以）

```js
export default {
  data() {
    return {
      post: {
        id: 1,
        title: 'My Journey with Vue'
      }
    }
  }
}
```
以下两种绑定是等价的
```html
<BlogPost v-bind="post" />
<BlogPost :id="post.id" :title="post.title" />
```

## 单向数据流动
prop参数传递是一种单向数据绑定
- 响应性：若父组件的实参发生变化，传到子组件中的形参也会相应的更新
- 单向：反过来，改变子组件中的形参不会影响父组件的实参

注意：父组件的每次更新都会使得子组件的形参更新，因此改变子组件中的形参没有意义（Vue会在控制台中发出警告）
```js
export default {
  props: ['foo'],
  created() {
    // ❌ warning, props are readonly!
    this.foo = 'bar'
  }
}
```

然而，经常会出现让人想在子组件中改变prop参数的情形
1. prop参数用作一个初始值来初始化子组件，随后子组件想将该prop属性当作一个普通的本地数据属性
    - 这种情形下，可以将prop参数赋值给另一个本地变量
    
```js
export default {
  props: ['initialCounter'],
  data() {
    return {
      // 这里只将prop参数 this.initialCounter 当作一个初始值
      // 将该参数值赋值给另一个变量，可以断开与prop参数的绑定
      counter: this.initialCounter
    }
  }
}
```

2. prop参数当作一个原始值，该值需要做一些转换
    - 这种情形下，可以使用计算属性
    
 ```js
export default {
  props: ['size'],
  computed: {
    // computed property that auto-updates when the prop changes
    normalizedSize() {
      return this.size.trim().toLowerCase()
    }
  }
}
```

注意，以上所说的（在子组件中）改变prop参数是指改变prop参数本身，不包括改变prop参数的内部属性。

换句话说，如果prop参数是Array或Object类型，那么是可以（在子组件中）改变prop参数的。这是因为JS的Array和Object是引用传递，如果Vue想防止这种深层次的改变会有很大的开销，因此不会发生警告，需要作者自行避免。

这种深层次改变会造成不好的结果：子组件会在父组件的不知情的情况下，改变父组件的状态，可能会触发其它意料之外的更新（尤其是在该状态是响应式状态的情况下）。

最好的方法是：
- 避免这种改变，除非想故意使得父组件与子组件紧密地耦合。
- 子组件应该发出(emit)一个事件给父组件，让父组件来改变

## prop参数验证
组件可以指定对参数的要求
- 例如之前讲的参数的类型
- 若是要求没满足，Vue会在控制台发生警告（development模式下）
- 参数验证是在组件实例创建之前，不能使用组件的属性（例如`data`,`computed`）

```js
export default {
  props: {
    // 基本的类型验证
    //  (`null` 和 `undefined` 允许所有类型)
    propA: Number,
    // 多个允许的类型
    propB: [String, Number],
    // Required string
    propC: {
      type: String,
      required: true // 要求每个参数非空（默认是允许为空，若为空，其实为undefined）
    },
    // Number with a default value
    propD: {
      type: Number,
      default: 100  // 提供一个默认值，用于参数为空或参数的值为undefined
    },
```
      

```
    propE: {
      type: Object,
      // Object和Array类型的默认值需要从一个工厂函数(名称为default)获得
      // 该函数有一个参数
      default(rawProps) {
        return { message: 'hello' }
      }
    },
    // 自定义一个“验证函数”，函数名称为validator, 该函数返回一个bool值来指示验证是否成功
    propF: {
      validator(value) {
        return ['success', 'warning', 'danger'].includes(value)
      }
    },
    // 若参数是Function类型, defalut函数即为默认的prop参数的值
    propG: {
      type: Function,
      default() {
        return 'Default function'
      }
    }
  }
}
```

Runtime类型验证：
- 数据类型可以是以下几种JS原生的Constructor
`String`, `Number`, `Boolean`, `Array`, `Object`, `Date`, `Function`, `Symbol`
- 也可以是自定义的constructor(或class), 验证是基于 instanceof 执行

```js
class Person {
  constructor(firstName, lastName) {
    this.firstName = firstName
    this.lastName = lastName
  }
}
export default {
  props: {
    author: Person
  }
}
```

## Bool类型转换
布尔类型prop参数有特殊的转换规则，来模拟html原生的布尔类型的attribute

```js
export default {
  props: {
    disabled: Boolean
  }
}
```

```html
<!-- 等价于 :disabled="true" -->
<MyComponent disabled />

<!-- 等价于 :disabled="false" -->
<MyComponent />
```

# Events

# 插槽(Slot)
# Provide/inject
# Fallthrough Attributes

# 异步组件(Async Component)